## 하이퍼파라미터를 이용한 성능 최적화

하이퍼파라미터를 이용한 성능 최적화의 추가적인 방법으로 `배치 정규화`, `드롭아웃`, `조기 종료`가 있습니다.

먼저 배치 정규화부터 살펴보겠습니다.

### 배치 정규화를 이용한 성능 최적화
배치 정규화를 진행하기에 앞서 유사한 의미로 사용되는 용어들을 알아보겠습니다.

### 정규화

정규화(normalization)는 데이터 범위를 `사용자가 원하는 범위로 제한`하는 것을 의미합니다. 예를 들어 이미지 데이터는 픽셀 정보로 0~255 사이의 값을 갖는데, 이를 255로 나누면 0~1.0 사이의 값을 갖게 됩니다.

![](../Static/462.jpg)

정규화는 각 특성 범위(scale)를 조정한다는 의미로 `특성 스케일링(feature scaling)`이라고도 합니다. 스케일 조정을 위해 `MinMaxScaler()` 기법을 사용하므로 수식은 다음과 같습니다.

![](../Static/fn2-3.jpg)

### 규제화

규제화(regulatization)는 `모델 복잡도를 줄이기 위해 제약`을 두는 방법입니다. 이때 제약은 데이터가 네트워크에 들어가기 전에 `필터를 적용한 것`이라고 생각하면 됩니다. 예를 들어 다음 왼쪽 그림은 필터가 적용되지 않을 경우 모든 데이터가 네트워크에 투입되지만, 오른쪽 그림은 필터로 걸러진 데이터만 네트워크에 투입되어 빠르고 정확한 결과를 얻을 수 있습니다.

![](../Static/463_1.jpg)

규제를 이용하여 모델 복잡도를 줄이는 방법은 다음과 같습니다.

* 드롭아웃
* 조기 종료

### 표준화
표준화(standardization)는 기존 데이터를 평균은 0, 표준편차는 1인 형태의 데이터로 만드는 방법입니다. 다른 표현으로 표준화 스칼라(standard scaler) 혹은 z-스코어 정규화(z-score normalization)라고도 합니다.

![](../Static/463_2.jpg)

평균을 기준으로 얼마나 떨어져 있는지 살펴볼 때 사용합니다. 보통 데이터 분포가 가우시안 분포를 따를 때 유용한 방법으로 다음 수식을 사용합니다.

![](../Static/fn2-4.jpg)

### 배치 정규화

배치 정규화(batch normalization)은 데이터 분포가 안정되어 학습 속도를 높일 수 있습니다.

배치 정규화는 기울기 소멸(gradient vanishing)이나 기울기 폭발(gradient exploding) 같은 문제를 해결하기 위한 방법입니다. 일반적으로 기울기 소멸이나 폭발 문제를 해결하기 위해 손실 함수로 `렐루(ReLU)` 를 사용하거나 초깃값 튜닝, `학습률(learning rate)`등을 조정합니다.

* 기울기 소멸 : 오차 정보를 역전파시키는 과정에서 기울기가 급격히 0에 가까워져 학습이 되지 않는 현상입니다.

* 기울기 폭발 : 학습 과정에서 기울기가 급격히 커지는 현상입니다.

![](../Static/465_1.jpg)

배치 정규화가 소개된 논문에 따르면 기울기 소멸과 폭발 원인은 내부 공변량 변화(internal covariance shift)때문인데, 이것은 네트워크의 각 층마다 활성화 함수가 적용되면서 입력 값들의 분포가 계속 바뀌는 현상을 의미합니다. 따라서 분산된 분포를 정규분포로 만들기 위해 표준화와 유사한 방식을 `미니 배치(mini-batch)`에 적용하여 평균은 0으로, 표준편차는 1로 유지하도록 하며 수식은 다음과 같습니다.

![](../Static/465_2.jpg)

1. 미니 배치의 평균을 구합니다.

2. 미니 배치의 분산과 표준편차를 구합니다.

3. 정규화를 수행합니다.

4. 스케일(scale)을 조정합니다.

따라서 매 단계마다 활성화 함수를 거치면서 데이터셋 분포가 일정해지기 때문에 속도를 향상시킬 수 있지만 다음과 같은 단점도 있습니다.

1. 배치 크기가 작을 때는 정규화 값이 기존 값과 다른 방향으로 훈련될 수 있습니다. 예를 들어 분산이 0 이면 정규화 자체가 안 되는 경우가 생길 수 있습니다.

2. RNN 네트워크 계층별로 미니 정규화를 적용해야 하기 때문에 모델이 더 복잡해지면서 비효율적일 수 있습니다.

따라서 이러한 문제들을 해결하기 위한 가중치 수정, 네트워크 구성 변경 등을 수행하지만, 무엇보다 중요한 것은 배치 정규화를 적용하면 적용하지 않았을 때보다 성능이 좋아지기 때문에 많이 사용됩니다.